ELEC5305 PROJECT MAIN CODE

In [3]:
'''
1. Pre - Requisite
Install all Library
'''
# 1) Upgrade core tooling inside THIS kernel
%pip install --upgrade pip setuptools wheel --user

# 2) Clean install of audio deps (librosa works with Py3.12 now via numba>=0.61)
%pip install librosa==0.11.0 numba==0.61.2 llvmlite soxr soundfile ffmpeg-python matplotlib tqdm pandas numpy --user

!pip install pandas numpy librosa torchaudio tqdm ffmpeg-python matplotlib soundfile --user
import os, sys, json, random, subprocess, shutil
from pathlib import Path
import numpy as np
import pandas as pd
import librosa, librosa.display
import soundfile as sf
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import Audio, display

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)

   ---------------------------------------- 0/2 [wheel]
   ---------------------------------------- 0/2 [wheel]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- --------------

Note: you may need to restart the kernel to use updated packages.


1) Config & paths

In [5]:
'''
Load SoccerNet Dataset
'''
# Run this in a notebook cell
%pip install SoccerNet --upgrade --user
from pathlib import Path
from SoccerNet.Downloader import SoccerNetDownloader

# Point to a local folder where SoccerNet will be cached
SN_ROOT = Path("./SoccerNet")   # or an absolute path on your machine

# Initialize downloader
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory=str(SN_ROOT))

# Example: download games with events and video (low-res)
# This requires free SoccerNet credentials (username/password) from https://www.soccer-net.org/data
mySoccerNetDownloader.downloadGames(files=["1_720p.mkv", "Labels-v2.json"], split=["train", "valid"], verbose=True)


  Using cached SoccerNet-0.1.62-py3-none-any.whl.metadata (13 kB)
  Using cached scikit_video-1.1.11-py2.py3-none-any.whl.metadata (1.1 kB)
  Using cached google_measurement_protocol-1.1.0-py2.py3-none-any.whl.metadata (845 bytes)
  Using cached pycocoevalcap-1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached boto3-1.40.30-py3-none-any.whl.metadata (6.7 kB)
  Using cached botocore-1.40.30-py3-none-any.whl.metadata (5.7 kB)
  Using cached s3transfer-0.14.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached InquirerPy-0.3.4-py3-none-any.whl.metadata (8.1 kB)
Using cached SoccerNet-0.1.62-py3-none-any.whl (86 kB)
Using cached boto3-1.40.30-py3-none-any.whl (139 kB)
Using cached botocore-1.40.30-py3-none-any.whl (14.0 MB)
Using cached s3transfer-0.14.0-py3-none-any.whl (85 kB)
Using cached google_measurement_protocol-1.1.0-py2.py3-none-any.whl (5.9 kB)
Using cached InquirerPy-0.3.4-py3-none-any.whl (67 kB)
Using cached pycocoevalcap-1.2-py3-none-any.whl (104.3 MB)
Using cached scikit_vid

password required for SoccerNet\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\1_720p.mkv


password required for SoccerNet\england_epl\2014-2015\2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal\1_720p.mkv


password required for SoccerNet\england_epl\2014-2015\2015-02-21 - 18-00 Swansea 2 - 1 Manchester United\1_720p.mkv


password required for SoccerNet\england_epl\2014-2015\2015-02-22 - 19-15 Southampton 0 - 2 Liverpool\1_720p.mkv


...

password required for SoccerNet\spain_laliga\2016-2017\2017-02-18 - 18-15 Real Madrid 2 - 0 Espanyol\1_720p.mkv


password required for SoccerNet\spain_laliga\2016-2017\2017-04-23 - 21-45 Real Madrid 2 - 3 Barcelona\1_720p.mkv


password required for SoccerNet\spain_laliga\2016-2017\2017-04-29 - 17-15 Real Madrid 2 - 1 Valencia\1_720p.mkv


password required for SoccerNet\spain_laliga\2016-2017\2017-04-29 - 21-45 Espanyol 0 - 3 Barcelona\1_720p.mkv


password required for SoccerNet\spain_laliga\2016-2017\2017-05-14 - 21-00 Las Palmas 1 - 4 Barcelona\1_720p.mkv


In [ ]:
'''
Setup Path and Parameters
'''
from pathlib import Path
import pandas as pd, numpy as np, json, random, subprocess, torch, torchaudio
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import Audio, display

SN_ROOT   = Path("./SoccerNet")           # where SoccerNetDownloader saved data
WORK_DIR  = Path("./sn_audio_work")
AUDIO_DIR = WORK_DIR / "full_match_audio_wav"
CLIPS_DIR = WORK_DIR / "clips"
FEAT_DIR  = WORK_DIR / "mel64"
for p in [WORK_DIR, AUDIO_DIR, CLIPS_DIR, FEAT_DIR]: 
    p.mkdir(parents=True, exist_ok=True)

TARGET_SR   = 16000
TARGET_LABELS = None     # Fill all 17 Labels
WINDOWS     = [15, 30]   # 15- and 30-sec windows
RANDOM_SEED = 1337

torchaudio.set_audio_backend("soundfile")    # safer on Windows
random.seed(RANDOM_SEED)


C:\Users\CYBORG 15\AppData\Local\Temp\ipykernel_33332\1006193538.py:24: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")    # safer on Windows


In [21]:
from pathlib import Path

# 🔁 EDIT THIS to your real folder
SN_ROOT = Path(r"C:/Users/CYBORG 15/Documents/GitHub/elec5305-project-520038655/SoccerNet")

print("SN_ROOT:", SN_ROOT)
print("Exists? ->", SN_ROOT.exists())


SN_ROOT: C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet
Exists? -> True


In [22]:
from pathlib import Path

candidates = list(SN_ROOT.rglob("Labels-v2.json")) + list(SN_ROOT.rglob("Labels.json"))
print("Found label files:", len(candidates))
for p in candidates[:5]:
    print(" •", p)


Found label files: 400
 • C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\Labels-v2.json
 • C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2014-2015\2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal\Labels-v2.json
 • C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2014-2015\2015-02-21 - 18-00 Swansea 2 - 1 Manchester United\Labels-v2.json
 • C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2014-2015\2015-02-22 - 19-15 Southampton 0 - 2 Liverpool\Labels-v2.json
 • C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2014-2015\2015-04-11 - 19-30 Burnley 0 - 1 Arsenal\Labels-v2.json


In [ ]:
# --- Parse labels + pair with the correct video (per-half) ---

from pathlib import Path
import json
import pandas as pd
from tqdm import tqdm

def find_label_files(root: Path):
    """Find SoccerNet label files under root (v2 preferred, fallback to legacy)."""
    files = list(root.rglob("Labels-v2.json"))
    if not files:
        files = list(root.rglob("Labels.json"))
    if files:
        print(f"✅ Found {len(files)} label files (showing up to 3):")
        for f in files[:3]:
            print("   •", f)
    else:
        print(f"⚠️ No label files found under: {root}")
    return files

def find_half_videos(match_dir: Path):
    """
    Return dict mapping half->Path and 'any' fallback:
      {'1': Path|None, '2': Path|None, 'any': Path|None}
    Tries common names first, then any mkv/mp4 in folder.
    """
    half_map = {1: None, 2: None}
    any_vid = None
    preferred = [
        ("1_720p.mkv", 1), ("2_720p.mkv", 2),
        ("1.mkv", 1),     ("2.mkv", 2),
        ("1.mp4", 1),     ("2.mp4", 2),
    ]
    for name, half in preferred:
        p = match_dir / name
        if p.exists():
            half_map[half] = p
            if any_vid is None:
                any_vid = p
    if any_vid is None:
        for ext in (".mkv", ".mp4", ".m4v", ".mov"):
            found = list(match_dir.glob(f"*{ext}"))
            if found:
                any_vid = found[0]
                break
    return {"1": half_map[1], "2": half_map[2], "any": any_vid}

def _parse_time_mmss(s: str):
    # Supports "HH:MM:SS" or "MM:SS"
    parts = s.split(":")
    try:
        if len(parts) == 3:
            hh, mm, ss = parts
            return int(hh)*3600 + int(mm)*60 + float(ss)
        if len(parts) == 2:
            mm, ss = parts
            return int(mm)*60 + float(ss)
    except Exception:
        return None
    return None

def load_labels_with_half(labels_path: Path):
    """
    Read label JSON and return list of dicts:
      {'time_s': float, 'label': str, 'half': int|0}
    Half is inferred from 'half' or 'gameTime' if present.
    Time is taken from 'position' (seconds) or parsed from 'gameTime'.
    """
    try:
        with open(labels_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except Exception as e:
        print(f"⚠️ Failed to read JSON: {labels_path} | {e}")
        return []

    ann = data.get("annotations") or data.get("events") or data.get("labels") or []
    events = []
    for ev in ann:
        lab = ev.get("label") or ev.get("event") or ev.get("type")
        if not lab:
            continue

        # infer half
        half = ev.get("half")
        if half not in (1, 2):
            gt = ev.get("gameTime")
            if isinstance(gt, str) and " - " in gt:
                try:
                    half = int(gt.split(" - ")[0].strip())
                except Exception:
                    half = None

        # infer time (seconds from half start)
        pos = ev.get("position")
        t = None
        if isinstance(pos, (int, float)):
            t = float(pos)
        elif isinstance(pos, str):
            t = _parse_time_mmss(pos)

        if t is None:
            gt = ev.get("gameTime")
            if isinstance(gt, str) and " - " in gt:
                t = _parse_time_mmss(gt.split(" - ")[1].strip())

        if t is None:
            continue

        events.append({"time_s": t, "label": lab, "half": (half if half in (1, 2) else 0)})
    return events

# ---- Build dataframe ----

rows = []
label_files = find_label_files(SN_ROOT)
print("📄 Label files found:", len(label_files))

for lf in tqdm(label_files, desc="Index matches (per-half)"):
    match_dir = lf.parent
    vids = find_half_videos(match_dir)           # {'1': Path|None, '2': Path|None, 'any': Path|None}
    evs  = load_labels_with_half(lf)
    if not evs:
        continue

    for e in evs:
        # choose video by half when available, else fallback to any
        video = vids[str(e["half"])] if e["half"] in (1, 2) else vids["any"]
        if video is None:
            # no video in this match directory → skip (audio pipeline needs a video/audio file)
            continue

        rows.append({
            "split": match_dir.parent.name,   # train/valid/test
            "match_dir": str(match_dir),
            "labels_json": str(lf),
            "video_path": str(video),
            "event_time_s": float(e["time_s"]),
            "event_label": e["label"],
            "half": int(e["half"])
        })

if not rows:
    raise RuntimeError(
        "No events paired to videos.\n"
        "- If labels exist but videos don't, your account likely has labels-only access.\n"
        "- Otherwise verify SN_ROOT and that video halves (1/2) are present."
    )

df_events = pd.DataFrame(rows).drop_duplicates(
    subset=["match_dir", "video_path", "event_time_s", "event_label"]
).reset_index(drop=True)

print("✅ Indexed events:", len(df_events))
print("🎥 Unique videos:", df_events['video_path'].nunique())
ALL_LABELS = sorted(df_events["event_label"].unique().tolist())
print("🏷️ Labels found:", ALL_LABELS)
print("🧮 Total classes:", len(ALL_LABELS))

# Use all labels present in your split
TARGET_LABELS = set(ALL_LABELS)



=== Trying ['1.mkv', '2.mkv', 'Labels-v2.json'] on split=['valid'] ===
password required for C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2014-2015\2015-04-11 - 19-30 Burnley 0 - 1 Arsenal\1.mkv
password required for C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2014-2015\2015-04-11 - 19-30 Burnley 0 - 1 Arsenal\2.mkv
C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2014-2015\2015-04-11 - 19-30 Burnley 0 - 1 Arsenal\Labels-v2.json already exists
password required for C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2015-2016\2015-08-30 - 18-00 Swansea 2 - 1 Manchester United\1.mkv
password required for C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2015-2016\2015-08-30 - 18-00 Swansea 2 - 1 Manchester United\2.mkv
C:\Users\CYBORG 15\Documents\GitHub\elec5305-project-520038655\SoccerNet\england_epl\2